<div style="width:100%; background-color: #000041"><a target="_blank" href="http://university.yugabyte.com"><img src="assets/YBU_Logo.png" /></a></div><br>

> **YugabyteDB YSQL Development**
>
> Enroll for free at [Yugabyte University](https://university.yugabyte.com/courses/yugabytedb-ysql-development).
>
<br>
This notebook file is:

`06_Geo-partitioning.ipynb`



# Advanced language features
In this notebook, you'll see how YSQL implements table partitioning and extends this using tablespaces so that developers can build geo-distributed applications that support row-level, geo-partitioning.

## 🛠️ Requirements
Here are the requirements for this notebook:
- ✅ Create the notebook variables in `01_Lab_Setup.ipynb`, which you previously did
- ✅ Create the `db_ybu` database, which you previously did
- ☑️ Import the notebook variables, *which you must do next*
- ☑️ Connect to the `db_ybu` database, *which you must do next*
- ☑️ Complete the following sections
  -  Table Partitioning
  -  Tablespaces and Row-Level Geo-Partitioning



### Select your notebook kernel
- In the Notebook toolbar, click **Select Kernel**.
<br>
<img width=50% src="assets/01_01_Select_Kernel_Toolbar.png" />

- Next, in the dropdown, select **Python 3.12** or higher.
<br>
<img width=50% src="assets/01_02_Select_Kernel_Dropdown.png" />

That's it!


## 👣 Setup steps
Here are the steps to setup this lab:
- Import the notebook variables
- Connect to `db_ybu` database
- Load the SQL Magic extension for the connection
- Create the prepared statements

### 👇 Import the notebook variables and style the notebook

> 👉 **IMPORTANT!** 👈
> 
> Do **NOT** skip running the following cells. 
>
> The following Python cell reads the stored variables created in the `01_Lab_Setup.ipynb` notebook. To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.  The cell after that styles the notebook.

👇 👇 👇 

In [ ]:
# Use %store -r to read 01_Lab_Setup variables
%store -r 
%config SqlMagic.autopandas=False
# %config SqlMagic.named_parameters=True
%config SqlMagic.displaylimit=30
%config SqlMagic.displaycon=False

**Update the styling of the notebook.**

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

## Connect to the `db_ybu` database
Run all the cells in this section:
- Connect using Python and PostgreSQL driver
- Load the SQL magic extension
- Create the prepared statements

### Connect using Python and PostgreSQL driver

In [ ]:
# connect use Python 3.12.1
import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine

db_host = NB_HOST_IPv4_01
db_name = NB_DB_NAME

connection_str = 'postgresql+psycopg2://yugabyte@'+db_host+':5433/'+db_name

engine = create_engine(connection_str)

### Load the SQL magic extension

In [ ]:
%reload_ext sql

# SQL magic for python connection string
%sql {connection_str}

### Create the prepared statements

> 👉 **IMPORTANT!** 👈
>   
> In order to create the prepared statements for the SQL magic connection, you must run the following cell!!!
> 
> **Do not skip this step**.
> 

In [ ]:
#%% python, but prepared statements as sql magic
if (NB_GITPOD_WORKSPACE_URL is None):
    stmt = %sql select fn_yb_create_stmts()
else:
    WORKSPACE_URL = NB_GITPOD_WORKSPACE_URL.replace('https://','https://7000-')
    stmt = %sql select fn_yb_create_stmts(:WORKSPACE_URL)

print(stmt)

Confirm that the following query returns a count of 3 (for three prepared statements).

In [ ]:
%%sql 
select count(*) from pg_prepared_statements where 1=1 and name in ('stmt_util_metrics_snap_tablet','stmt_util_metrics_snap_table','stmt_util_metrics_snap_reset')

---

## Table Partitioning
Partitioning is another term for physically dividing large tables in YugabyteDB into smaller, more manageable tables to improve performance. Typically, tables with columns containing timestamps are subject to partitioning because of the historical and predictable nature of their data.

Because partitioned tables do not appear nor act differently from the original table, applications accessing the database are not always aware of table partitioning.

YSQL supports the following types of partitioning:

- *Range partitioning*<br> when a table is partitioned into ranges defined by one or more key columns. In this case, the ranges of values assigned to partitions do not overlap.

- *List partitioning*<br> when a table is partitioned via listing key values to appear in each partition.

- *Hash partitioning*<br> when a table is partitioned by specifying a modulus and remainder for each partition.

### q1 | Table partitioning by range
In this section of the notebook, you will:
- Create tables with a DDL script
- Load data with a DML script
- Verify the creation of tables and data
- View the DDL for `order_changes`

##### Create tables, load data, and review relations
Run the following cell to execute the DDL and DML scripts using `ysqlsh`.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME" "$NB_NOTEBOOK_DATA_FOLDER" "$NB_DATA_DDL_FILE_3" "$NB_DATA_DML_FILE_3"   # tbl_order_changes
YB_PATH_BIN=${1}
DB_NAME=${2}
DATA_FOLDER=${3}
DATA_DDL_FILE=${4}
DATA_DML_FILE=${5}

ORDER_DDL_PATH=${DATA_FOLDER}/${DATA_DDL_FILE}
ORDER_DML_PATH=${DATA_FOLDER}/${DATA_DML_FILE}

echo $ORDER_DDL_PATH
echo $ORDER_DML_PATH

cd $YB_PATH_BIN

# DDL file
./ysqlsh -d ${DB_NAME} -f ${ORDER_DDL_PATH} >&/dev/null
sleep 1;

# DML file
./ysqlsh -d ${DB_NAME} -f ${ORDER_DML_PATH} >&/dev/null
sleep 1;

Describe the tables.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME" "$NB_NOTEBOOK_DATA_FOLDER" "$NB_DATA_DDL_FILE_3" "$NB_DATA_DML_FILE_3"   # tbl_order_changes
YB_PATH_BIN=${1}
DB_NAME=${2}

cd $YB_PATH_BIN


# Describe tables
./ysqlsh -d ${DB_NAME} -c "\d public.tbl_order_*"

There is one table that has several table partitions:
- `tbl_order_changes`
  - `tbl_order_changes_prtn_2023_01`
  - `tbl_order_changes_prtn_2023_02`
  - `tbl_order_changes_prtn_2023_03`
  - `tbl_order_changes_prtn_2023_04`
  - 
  The table partitions uses the following PK definition:
  -  `PRIMARY KEY, lsm (user_id HASH, change_date DESC)`
  
  The partition key is user_id, and the clustering key is `change_date` in descending order. 

Experiment with the following query to see how rows exist in the table partitions. The `tableoid::regclass` column details the partition of the row. 

In [ ]:
%%sql
select '' _
  , tableoid::regclass
  , user_id 
  , account_id
  , change_date
  , description
from 
  tbl_order_changes
-- tbl_order_changes_prtn_2023_01
-- tbl_order_changes_prtn_2023_02
-- tbl_order_changes_prtn_2023_03
-- tbl_order_changes_prtn_2023_04
-- tbl_order_changes_prtn_default
;

#### View the Table details in the YB-Master web ui
In your web browser, open the following URL.

In [ ]:
#%% python, but prepared statements as sql magic
# THIS_TABLE_NAME='tbl_order_changes'
THIS_TABLE_NAME='tbl_order_changes_prtn_2023_01'
THIS_SCHEMA_NAME = 'public'
DB_NAME = NB_DB_NAME

## Comment out if local
view_gitpod_url = %sql select fn_get_table_id_url(:NB_YB_MASTER_HOST_GITPOD_URL,7000,:DB_NAME,:THIS_SCHEMA_NAME,:THIS_TABLE_NAME ) as view_gitpod_url
print (view_gitpod_url)

## Uncomment if local
# view_local_url = %sql select fn_get_table_id_url(:NB_HOST_IPv4_01,7000,:DB_NAME,:THIS_SCHEMA_NAME,:THIS_TABLE_NAME ) as view_local_url
# print (view_local_url)

#### q1a | View the data distribution 

In [ ]:
%%sql /* explain plan */
select '' _
  , yb_hash_code( user_id::int, change_date::date ) as pk_hash_code
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( user_id::int, change_date::date ),'hash_split: [0x0000, 0x5555)'::text) as col_0x0000_0x5555
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( user_id::int, change_date::date ),'hash_split: [0x5555, 0xAAAA)'::text) as col_0x5555_0xAAAA
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( user_id::int, change_date::date ),'hash_split: [0xAAAA, 0xFFFF)'::text) as col_0xAAAA_0xFFFF
  , tableoid::regclass
  , user_id 
 -- , account_id
  , change_date
 -- , description
from 
tbl_order_changes
-- tbl_order_changes_prtn_2023_01
-- tbl_order_changes_prtn_2023_02
-- tbl_order_changes_prtn_2023_03
-- tbl_order_changes_prtn_2023_04
-- tbl_order_changes_prtn_default
where 1=1 
-- and user_id=1
-- and change_date >= '2023-01-01' and  change_date <= '2023-01-31'
-- and change_date >= '2023-02-01' and  change_date <= '2023-02-28'
-- and change_date >= '2023-03-01' and  change_date <= '2023-03-31'
and change_date >= '2023-04-01' and  change_date <= '2023-04-30'
;

The query shows that consistent hash sharding of the data within the table partition, meaning that each table partition has tablet leaders among the three nodes in this cluster.

To learn more about how this affects query performance, run the following Explain Plan.

In [ ]:
%%sql /* explain plan */
execute stmt_util_metrics_snap_reset;
explain (analyze, dist) 
select '' _
  , yb_hash_code( user_id::int, change_date::date ) as pk_hash_code
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( user_id::int, change_date::date ),'hash_split: [0x0000, 0x5555)'::text) as col_0x0000_0x5555
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( user_id::int, change_date::date ),'hash_split: [0x5555, 0xAAAA)'::text) as col_0x5555_0xAAAA
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( user_id::int, change_date::date ),'hash_split: [0xAAAA, 0xFFFF)'::text) as col_0xAAAA_0xFFFF
  , tableoid::regclass
  , user_id 
 -- , account_id
  , change_date
 -- , description
from 
tbl_order_changes
-- tbl_order_changes_prtn_2023_01
-- tbl_order_changes_prtn_2023_02
-- tbl_order_changes_prtn_2023_03
-- tbl_order_changes_prtn_2023_04
-- tbl_order_changes_prtn_default
where 1=1 
-- and user_id=1
-- and change_date >= '2023-01-01' and  change_date <= '2023-01-31'
-- and change_date >= '2023-02-01' and  change_date <= '2023-02-28'
-- and change_date >= '2023-03-01' and  change_date <= '2023-03-31'
and change_date >= '2023-04-01' and  change_date <= '2023-04-20'
;

#### q1b | Explain Plan (above ^^)
```
Append (cost=0.00..870.00 rows=1000 width=144) (actual time=2.481..2.917 rows=4 loops=1)
  -> Seq Scan on tbl_order_changes_prtn_2023_04 (cost=0.00..865.00 rows=1000 width=144) (actual time=2.481..2.916 rows=4 loops=1)
        Remote Filter: ((change_date >= '2023-04-01'::date) AND (change_date <= '2023-04-20'::date))
        Storage Table Read Requests: 1
        Storage Table Read Execution Time: 1.379 ms
Planning Time: 0.250 ms
Execution Time: 3.025 ms
Storage Read Requests: 1
Storage Read Execution Time: 1.379 ms
Storage Write Requests: 0
Catalog Read Requests: 1
Catalog Read Execution Time: 1.749 ms
Catalog Write Requests: 0
Storage Flush Requests: 0
Storage Execution Time: 3.129 ms
Peak Memory Usage: 24 kB
```

Some observations:
-   `Seq Scan` is for the table partition, `tbl_order_changes_prtn_2023_04`
-  The `Remote Filter` indicates a pushdown filter to the tablets of the table partition, which is an optimization.


In [ ]:
%%sql
execute stmt_util_metrics_snap_table;

##### q1c | Metrics (above ^^)
This query reads from the table partition tablet leaders.

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_order_changes_prtn_2023_04 link_table_id tablet_id_unq_1	 | 2 | 	2 |
| db_ybu tbl_order_changes_prtn_2023_04 link_table_id tablet_id_unq_2	| 3	| 4 |
| db_ybu tbl_order_changes_prtn_2023_04 link_table_id tablet_id_unq_3	| 2	| 2 |


### q2 | Indexed Relations

When you create an index on the table with table partitions, YugabyteDB creates a matching index on the existing table partitions and any future partitions. An index or unique constraint on a partitioned table is virtual in the same way that the partitioned table is: the actual data lives in the child indexes.

Create the index on the table for `change_date`, the predicate reference in the previous query.

In [ ]:
%%sql
drop index if exists idx_order_changes_range;
create index if not exists idx_order_changes_range on tbl_order_changes (change_date desc);

Now check the explain plain to see how the query planner uses the index.

In [ ]:
%%sql /* explain plan */
execute stmt_util_metrics_snap_reset;
explain (analyze, dist) 
select '' _
  , yb_hash_code( user_id::int, change_date::date ) as pk_hash_code
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( user_id::int, change_date::date ),'hash_split: [0x0000, 0x5555)'::text) as col_0x0000_0x5555
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( user_id::int, change_date::date ),'hash_split: [0x5555, 0xAAAA)'::text) as col_0x5555_0xAAAA
  , fn_find_hash_code_in_partition_hex_range(yb_hash_code( user_id::int, change_date::date ),'hash_split: [0xAAAA, 0xFFFF)'::text) as col_0xAAAA_0xFFFF
  , tableoid::regclass
  , user_id 
 -- , account_id
  , change_date
 -- , description
from 
tbl_order_changes
-- tbl_order_changes_prtn_2023_01
-- tbl_order_changes_prtn_2023_02
-- tbl_order_changes_prtn_2023_03
-- tbl_order_changes_prtn_2023_04
-- tbl_order_changes_prtn_default
where 1=1 
-- nd user_id=1
-- and change_date >= '2023-01-01' and  change_date <= '2023-01-31'
-- and change_date >= '2023-02-01' and  change_date <= '2023-02-28'
-- and change_date >= '2023-03-01' and  change_date <= '2023-03-31'
and change_date >= '2023-04-01' and  change_date <= '2023-04-20'
;

#### q2a | Explain Plan (above ^^)

```
Append (cost=0.00..12.90 rows=10 width=144) (actual time=9.757..10.300 rows=4 loops=1)
  -> Index Scan using tbl_order_changes_prtn_2023_04_change_date_idx on tbl_order_changes_prtn_2023_04 (cost=0.00..12.85 rows=10 width=144) (actual time=9.756..10.298 rows=4 loops=1)
        Index Cond: ((change_date >= '2023-04-01'::date) AND (change_date <= '2023-04-20'::date))
        Storage Table Read Requests: 1
        Storage Table Read Execution Time: 1.659 ms
        Storage Index Read Requests: 1
        Storage Index Read Execution Time: 3.815 ms
Planning Time: 17.817 ms
Execution Time: 10.423 ms
Storage Read Requests: 2
Storage Read Execution Time: 5.474 ms
Storage Write Requests: 0
Catalog Read Requests: 30
Catalog Read Execution Time: 22.190 ms
Catalog Write Requests: 0
Storage Flush Requests: 0
Storage Execution Time: 27.665 ms
Peak Memory Usage: 24 kB
```

Some observations about the Query Plan:
- `Index Scan` indicates that the query utilizes and index.
  - The index name differs from the index previously created, `idx_order_changes_range`
  - YugabyteDB creates the table partition index using it's own naming convention, `tbl_order_changes_prtn_2023_04_change_date_idx`, where `idx` is a suffix.
- `Index Cond` applies the filter.

In [ ]:
%%sql
execute stmt_util_metrics_snap_table;

##### q2b | Metrics (above ^^)
Here, the index partition is accessed once, and all three related tables partitions are then accessed.

| row_name| 	rocksdb_number_db_seek | 	rocksdb_number_db_next | 
|--|--|--|
| db_ybu tbl_order_changes_prtn_2023_04 link_table_id tablet_id_unq_1	 | 2 | 	none |
| db_ybu tbl_order_changes_prtn_2023_04 link_table_id tablet_id_unq_2	| 1	| none |
| db_ybu tbl_order_changes_prtn_2023_04 link_table_id tablet_id_unq_3	| 1	| none |
| db_ybu tbl_order_changes_prtn_2023_04_change_date_idx  link_table_id tablet_id_unq_4	| 1	| 4 |


Question:
- Why is there only one index partition for the partitioned table?

Answer:
- The index uses range sharding! A tablet for a user table or index with a partition key using range starts with just one tablet leader on a single node, and as it grows will split into two tablets, and so on.

Take a look at the definition below:
- `USING lsm (change_date DESC)`

In [ ]:
%%sql
select pg_get_indexdef('tbl_order_changes_prtn_2023_04_change_date_idx':: regclass);_range':: regclass);

To see the related indexes, run the following. Note that reference to `Partition of: idx_order_changes_range `. This is the name of the range index that was created earlier.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME" "$NB_NOTEBOOK_DATA_FOLDER" "$NB_DATA_DDL_FILE_3" "$NB_DATA_DML_FILE_3"   # tbl_order_changes
YB_PATH_BIN=${1}
DB_NAME=${2}

cd $YB_PATH_BIN

# Describe tables
./ysqlsh -d ${DB_NAME} -c "\d+ public.tbl_order_changes_prtn_*_idx"

#### Additional considerations

You can add or remove table partitions, as well as attach and detach partitions. Here are some other considerations:
- The primary key for a partitioned table should always contain the partition key.
- If you choose to define row triggers, you do so on individual partitions instead of the partitioned table.
- Creating a foreign key reference on a partitioned table is not supported.
- A partition table inherits tablespaces from its parent.
- You cannot mix temporary and permanent relations in the same partition hierarchy.
- If you have a default partition in the partitioning hierarchy, you can add new partitions only if there is no data in the default partition that matches the partition constraint of the new partition.
  
  
To learn more about table partitioning, check out:

<a href target="_blank" href="https://docs.yugabyte.com/preview/explore/ysql-language-features/advanced-features/partitions">Table Partitioning</a> at docs.yugabyte.com.

---
# Tablespaces and Row-level Geo-Partitioning
YugabyteDB extends the concept of PostgreSQL tablespaces for a distributed database. In PostgreSQL, tablespaces allow administrators to specify specific tables and indexes should reside on disk, typically based on how users want to store and access the data. This control over data placement enables fine-grained performance tuning. You can, for example, place heavily accessed smaller tables and indexes on fast SSDs.

YSQL tablespaces re-purpose this concept for a geo-distributed deployment by allowing you to:
-  specify the number of replicas for a table or index,
-  how to distribute across a set of clouds, regions, and zones

Replicating and pinning tables in specific regions can lower read latency, improve resilience, and achieve compliance with data residency laws. 

For example, you can create duplicate indexes on the same column of a table and place these indexes close to users in different regions for fast access. Similarly, you can partition a master table and associate the partitions with different tablespaces to pin the data geographically.

The ability to control the placement of tables in a fine-grained manner provides the following advantages:

- Tables with critical information can have higher replication factor and increased fault tolerance compared to the rest of the data.
- Based on the access pattern, a table can be constrained to the region or zone where it's accessed most frequently.
- A table can have an index with an entirely different placement policy, thereby boosting the read performance without affecting the placement policy of the table itself.
- Coupled with table partitioning, tablespaces can be used to implement row-level geo-partitioning. In other words,  based on the values of certain columns in a given row, you can pin the rows of a table to different geo-locations.

## q3 | Tablespaces

Tablespaces are assigned repositories with assigned locations. The definition of tablespaces includes the number of replicas as well as placement blocks. Placement blocks are a cluster configuration.  Nodes in the cluster are assigned to placement blocks. Here is the configuration for this notebook

` --placement_info "cloud1.region1.zone1,cloud1.region2.zone2,cloud1.region3.zone3" `

They are arbitrarily named: cloud, region and zone.
- `cloud1.region1.zone1`
- `cloud1.region2.zone2`
- `cloud1.region3.zone3`
  
The definition of tablespace must align with placement black names in the cluster.


You can view the placement blocks per table. You can also see the placement blocks using the YB-Master UI.

In [ ]:
print (NB_YB_MASTER_HOST_GITPOD_URL)

**Masters**
|Server|RAFT Role|Uptime|Details|
|-|-|-|-|
|127.0.0.1:7000|	LEADER|	17:48:54|CLOUD: cloud1<br>REGION: region1<br>ZONE: zone1<br>UUID: 0d97797eb98b4327a9f0b0c8c8021dc2|
|127.0.0.2:7000|FOLLOWER|17:48:54|CLOUD: cloud1<br>REGION: region2<br>ZONE: zone2<br>UUID: a2576e3f7f324083bb9fdc1dd05e73d6|
|127.0.0.3:7000|	FOLLOWER|	17:48:54|CLOUD: cloud1<br>REGION: region3<br>ZONE: zone3<br>UUID: 24df5c170f664b5b91072b9f1f6708d9|


In [ ]:
%%sql
drop table if exists tbl_transactions;

`cloud1.region1.zone1` is for the US geographic region.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME" #\d+
YB_PATH_BIN=${1}
DB_NAME=${2}

cd $YB_PATH_BIN

#  can't do with sql magic 
# needs double quote esacpe for bash because of JSON
#
# create tablespace tblspace_us with (replica_placement=
#  '{"num_replicas": 1, 
#    "placement_blocks": [{
#       "cloud": "cloud1", "region": "region1", "zone": "zone1", "min_num_replicas": 1}]
#   }'
# );
DDL_TABLESPACE_US="drop tablespace if exists tblspace_us; create tablespace tblspace_us with (replica_placement='{"\"num_replicas"\": 1, "\"placement_blocks"\": [{ "\"cloud"\": "\"cloud1"\", "\"region"\": "\"region1"\", "\"zone"\": "\"zone1"\", "\"min_num_replicas"\": 1}]}');" 

echo $DDL_TABLESPACE_US | ./ysqlsh -d ${DB_NAME}

`cloud1.region2.zone2` is for the EU  geographic region.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME" #\d+
YB_PATH_BIN=${1}
DB_NAME=${2}

cd $YB_PATH_BIN

#  can't do with sql magic 
# needs double quote esacpe for bash because of JSON

DDL_TABLESPACE_EU="drop tablespace if exists tblspace_eu; create tablespace tblspace_eu with (replica_placement='{"\"num_replicas"\": 1, "\"placement_blocks"\": [{ "\"cloud"\": "\"cloud1"\", "\"region"\": "\"region2"\", "\"zone"\": "\"zone2"\", "\"min_num_replicas"\": 1}]}');" 

echo $DDL_TABLESPACE_EU | ./ysqlsh -d ${DB_NAME}




`cloud1.region3.zone3` is for the AP geographic region.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME" #\d+
YB_PATH_BIN=${1}
DB_NAME=${2}

cd $YB_PATH_BIN

#  can't do with sql magic 
# needs double quote esacpe for bash because of JSON

DDL_TABLESPACE_AP="drop tablespace if exists tblspace_ap; create tablespace tblspace_ap with (replica_placement='{"\"num_replicas"\": 1, "\"placement_blocks"\": [{ "\"cloud"\": "\"cloud1"\", "\"region"\": "\"region3"\", "\"zone"\": "\"zone3"\", "\"min_num_replicas"\": 1}]}');" 

echo $DDL_TABLESPACE_AP | ./ysqlsh -d ${DB_NAME}




Review the table spaces.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME"  

YB_PATH_BIN=${1}
DB_NAME=${2}

cd $YB_PATH_BIN

./ysqlsh -d ${DB_NAME} -c "\db+ tblspace_*"

#### q3a | Create Table Partitions

The partitions will determine the which rows are included with the value from `geo_location`. Since the partitioned table has the Partition property by LIST, not RANGE, only rows that contain the LIST value will be assigned to a partition.

In [ ]:
%%sql
drop table if exists tbl_transactions;
create table if not exists tbl_transactions (
  user_id       int not null,
  account_id    int not null,
  geo_partition text,
  account_type  text not null,
  amount        numeric not null,
  created_at    timestamp default now()
) partition by list (geo_partition)
;


Create the partition for the US region with the assigned tablespace.

In [ ]:
%%sql 
drop table if exists tbl_transactions_prtn_us;
create table tbl_transactions_prtn_us partition of tbl_transactions (
    user_id, 
    account_id, 
    geo_partition, 
    account_type, 
    amount, 
    created_at,
    primary key (user_id hash, account_id, geo_partition)
) for values in ('US') 
tablespace tblspace_us;

Create the partition for the EU region with the assigned tablespace.

In [ ]:
%%sql 
drop table if exists tbl_transactions_prtn_eu;
create table tbl_transactions_prtn_eu partition of tbl_transactions (
    user_id, 
    account_id, 
    geo_partition, 
    account_type, 
    amount, 
    created_at,
    primary key (user_id hash, account_id, geo_partition)
) for values in ('EU') 
tablespace tblspace_eu;

Create the partition for the AP region with the assigned tablespace.

In [ ]:
%%sql 
drop table if exists tbl_transactions_prtn_ap;
create table if not exists tbl_transactions_prtn_ap partition of tbl_transactions (
    user_id, 
    account_id, 
    geo_partition, 
    account_type, 
    amount, 
    created_at,
    primary key (user_id hash, account_id, geo_partition)
) for values in ('AP') 
tablespace tblspace_ap;

View the table in the YB-Master Web Ui.

In [ ]:
#%% python, but prepared statements as sql magic
THIS_TABLE_NAME = 'tbl_transactions'
THIS_SCHEMA_NAME = 'public'
DB_NAME = NB_DB_NAME

## Comment out if local
view_gitpod_url = %sql select fn_get_table_id_url(:NB_YB_MASTER_HOST_GITPOD_URL,7000,:DB_NAME,:THIS_SCHEMA_NAME,:THIS_TABLE_NAME ) as view_gitpod_url
print (view_gitpod_url)

## Uncomment if local
# view_local_url = %sql select fn_get_table_id_url(:NB_HOST_IPv4_01,7000,:DB_NAME,:THIS_SCHEMA_NAME,:THIS_TABLE_NAME ) as view_local_url
# print (view_local_url)

#### q3b | Add records to the transactions table
Data localization also has a role in performance as well. Keeping the data source closer to the client will reduce network latency, improving the response time of the database. Having an understanding of what data is needed where coupled with the ability to place data in particular location is an important tool in distributed sql systems.

In [ ]:
%%sql
insert into tbl_transactions select generate_series(1,65535), round(1000*random()), 'US', 'customer',  round(1000*random()), now();
insert into tbl_transactions select generate_series(65536,100000), round(1000*random()), 'EU', 'customer',  round(1000*random()), now();
insert into tbl_transactions select generate_series(100000, 150000), round(1000*random()), 'AP', 'customer',  round(1000*random()), now();

Verify the inserts for AP. Every table contains the `tableoid` system column. `tableoid` is the `oid` (object identifier) of the table to which the row belongs. While querying a partitioned table, the tableoid column returns the oid of the partition to which the row belongs. Using the `regclass` type, you can view the the name of the object.

In [ ]:
%%sql
select '' _
  , tableoid
  , tableoid::regclass
  , user_id
  , account_id
  , geo_partition
from tbl_transactions
where 1=1
  and geo_partition ='AP'
-- and geo_partition ='EU'
--  and geo_partition ='US'
limit 10;

##### View the explain plan and metrics

In this example, we are using Object Identifiers to locate the partition table that is associated with the row of data.

In [ ]:
%%sql /* explain plan */
execute stmt_util_metrics_snap_reset;
explain (analyze, dist) 
select '' _
  , tableoid::regclass
  , user_id
  , account_id
  , geo_partition
from tbl_transactions
where 1=1 
  -- and user_id = 1
  and geo_partition = 'US'
-- and user_id = 2
-- and geo_partition = 'EU'
-- and user_id = 3
-- and geo_partition = 'AP'
-- and yb_is_local_table(tableoid)
;

```
Append (actual time=3.112..125.423 rows=65535 loops=1)
  -> Seq Scan on public.tbl_transactions_prtn_us (actual time=3.112..120.394 rows=65535 loops=1)
        Output: ''::text, (tbl_transactions_prtn_us.tableoid)::regclass, tbl_transactions_prtn_us.user_id, tbl_transactions_prtn_us.account_id, tbl_transactions_prtn_us.geo_partition
        Remote Filter: (tbl_transactions_prtn_us.geo_partition = 'US'::text)
Planning Time: 0.255 ms
Execution Time: 128.625 ms
Peak Memory Usage: 24 kB
```

#### q3c | Explain Plan (above ^^)

A few observation about the query plan:
- The `Seq Scan` is for the table partition.
- `Remote Filter` indicates a pushdown

In [ ]:
%%sql
execute stmt_util_metrics_snap_table;

##### q3d | Metrics (above ^^)
Here, the query reads from one table partition and returns all rows.

| row name |	rocksdb_number_db_seek	| rocksdb_number_db_next	| rows_inserted |
|-|-|-|-|
| db_ybu tbl_transactions_prtn_us https://7000-yugabytedbu-ysqldevelop-ggq7agkk4rt.ws-us107.gitpod.io/table?id=00004000000030008000000000004167 f1cb4247efe84e12a9322731b2551c9f L	| 65599	| 131070	| None |

### q4 | Built-in Geo-partitioning helper functions
With built-in functions for geo-partitioning, you can simplify reads and writes.

|FUNCTION	|RETURN TYPE|	DESCRIPTION|
|-|-|-|
|`yb_is_local_table(oid)`|	`boolean`|	Returns whether the given `oid` is a table replicated only in the local region|
|`yb_server_region()`|	`varchar`|	Returns the region of the currently connected node|
|`yb_server_zone()`|	`varchar`|	Returns the zone of the currently connected node|
|`yb_server_cloud()`|	`varchar`|	Returns the cloud provider of the currently connected node|

The listener address shows the connection for the %%sql magic cells. By default, it is `127.0.0.1`. The YB-Master Server Web UI shows the placement association with each host.

In [ ]:
%%sql
show listen_addresses;

**Masters**
|Server|RAFT Role|Uptime|Details|
|-|-|-|-|
|127.0.0.1:7000|	LEADER|	17:48:54|CLOUD: cloud1<br>REGION: region1<br>ZONE: zone1<br>UUID: 0d97797eb98b4327a9f0b0c8c8021dc2|



In [ ]:
print (NB_YB_MASTER_HOST_GITPOD_URL)

Since the connection for all `%%sql` magic cells in this notebook is to `127.0.0.1`, the following query can utilize this information.  The `where` clause predicate indicates how:
- `yb_is_local_table(tableoid)`

The `tableoid` is for the local connection.

In [ ]:
%%sql
select '' _
   , yb_server_cloud()
   , yb_server_region()
   , yb_server_zone()
   , tableoid::regclass
   , user_id
   , account_id
   , geo_partition
from tbl_transactions
where 1=1 
and yb_is_local_table(tableoid)
limit 10;

The query show result for the `tableoid:tbl_transactions_prtn_us`.  All the rows are for the `US`. The reason for this is:
- `tbl_transactions_prtn_us` belongs to the table_space, `tblspace_us`
- `tblspace_us` has a placement block that is `cloud1.region1.zone1`
- `127.0.0.1` has the same placement,  `cloud1.region1.zone1`

View the query plan. This will show `Filter: yb_is_local_table(tableoid)`.

In [ ]:
%%sql /* explain plan */

explain (analyze, dist)
select '' _
  , yb_server_cloud()
  , yb_server_region()
  , yb_server_zone()
  , tableoid::regclass
  , user_id
  , account_id
  , geo_partition
from tbl_transactions
where 1=1 
--  and user_id = 1
-- and geo_partition = 'US'
-- and user_id = 2
-- and geo_partition = 'EU'
-- and user_id = 3
-- and geo_partition = 'AP'
and yb_is_local_table(tableoid)
limit 10
;

```
Limit (cost=0.00..1.01 rows=10 width=76) (actual time=1.595..1.934 rows=10 loops=1)
  -> Append (cost=0.00..101.50 rows=1000 width=76) (actual time=1.594..1.931 rows=10 loops=1)
        -> Seq Scan on tbl_transactions_prtn_us (cost=0.00..96.50 rows=1000 width=76) (actual time=1.594..1.929 rows=10 loops=1)
              Filter: yb_is_local_table(tableoid)
              Storage Table Read Requests: 1
              Storage Table Read Execution Time: 1.493 ms
Planning Time: 0.228 ms
Execution Time: 3.047 ms
Storage Read Requests: 1
Storage Read Execution Time: 1.493 ms
Storage Write Requests: 0
Catalog Read Requests: 1
Catalog Read Execution Time: 0.853 ms
Catalog Write Requests: 0
Storage Flush Requests: 0
Storage Execution Time: 2.346 ms
Peak Memory Usage: 56 kB
```

The `yb_is_local_table` built-in function. You can use this function to show all database objects associated with a tablespace. The following query shows that relations for any tablespaces related to the `local` connection.

In [ ]:
%%sql
select oid, relname from pg_class where yb_is_local_table(oid);

To confirm the behavior of the function, change `%sql` magic connection to use another host, in this case, host `127.0.0.1`.

In [ ]:
# connect use Python 3.7.9+
db_host=NB_HOST_IPv4_02
# db_host=NB_HOST_IPv4_03
db_name=NB_DB_NAME

connection_str='postgresql+psycopg2://yugabyte@'+db_host+':5433/'+db_name

Reload the connection for the `%%sql` magic cells.

In [ ]:
%reload_ext sql
%sql {connection_str}

Verify that there is a new host.

In [ ]:
NEW_HOST = %sql show listen_addresses;
print (NEW_HOST)

Now, rerun the query using the new connection on `127.0.0.2`.

In [ ]:
%%sql
select '' _
  , yb_server_cloud()
  , yb_server_region()
  , yb_server_zone()
  , tableoid::regclass
  , user_id
  , account_id
  , geo_partition
from tbl_transactions
where 1=1 
and yb_is_local_table(tableoid)
limit 10;

The results are for the `EU`. Review the explain plan to verify that the table partition is `tbl_transactions_prtn_eu`.

In [ ]:
%%sql /* explain plan */


explain (analyze, dist)
select '' _
  , yb_server_cloud()
  , yb_server_region()
  , yb_server_zone()
  , tableoid::regclass
  , user_id
  , account_id
  , geo_partition
from tbl_transactions
where 1=1 
--  and user_id = 1
-- and geo_partition = 'US'
-- and user_id = 2
-- and geo_partition = 'EU'
-- and user_id = 3
-- and geo_partition = 'AP'
and yb_is_local_table(tableoid)
limit 10
;

```
Limit (cost=0.00..1.09 rows=10 width=176) (actual time=2.508..2.755 rows=10 loops=1)
  -> Append (cost=0.00..109.00 rows=1000 width=176) (actual time=2.507..2.753 rows=10 loops=1)
        -> Seq Scan on tbl_transactions_prtn_eu (cost=0.00..104.00 rows=1000 width=176) (actual time=2.507..2.751 rows=10 loops=1)
              Filter: yb_is_local_table(tableoid)
              Storage Table Read Requests: 1
              Storage Table Read Execution Time: 2.336 ms
Planning Time: 0.404 ms
Execution Time: 3.927 ms
Storage Read Requests: 1
Storage Read Execution Time: 2.336 ms
Storage Write Requests: 0
Catalog Read Requests: 0
Catalog Write Requests: 0
Storage Flush Requests: 0
Storage Execution Time: 2.336 ms
Peak Memory Usage: 72 kB

```

The query plan confirms that the `127.0.0.2` connection uses the `tbl_transactions_prtn_eu` partition.
- `tbl_transactions_prtn_eu` belongs to the table_space, `tblspace_eu`
- `tblspace_eu` has a placement block that is `cloud1.region2.zone2`
- `127.0.0.2` has the same placement,  `cloud1.region2.zone2`

To learn more about the `yb_is_local_table` built-in function, check out:
- [Docs --> yb_is_local_table](https://docs.yugabyte.com/preview/api/ysql/exprs/geo_partitioning_helper_functions/func_yb_is_local_table/)

---
# 🌟🌟🌟🌟🌟🌟   All  done! 
In this notebook, you completed the following:
In this lab, you completed the following:
- Created a partitioned table consisting of table partitions
- Created tablespaces
- Created table partitions assigned to the tabelspaces
- Queried the tables using row-level geo-partitioning

